In [2]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# import kpi_formula_calc

import sys
sys.path.append("/home/blocktist/Projects/analytics/utils")

from util_kpi import *
from util_visualization import *

# from kpi_formula_calc import safediv

%load_ext autoreload
%autoreload 2

In [3]:
PATH_FILE_DATA = '/home/blocktist/Projects/dataset/Energy/energy_dataset.csv'

In [4]:
df_data = pd.read_csv(PATH_FILE_DATA)

In [37]:
# df_data.sample(10000).to_csv('/home/blocktist/Projects/analytics/sample_data/energy_dataset_sample.csv')

In [5]:
df_data.head(5)

,timestamp,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17,NaN,6436,26118,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16,NaN,5856,24934,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8,NaN,5454,23515,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2,NaN,5151,22642,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9,NaN,4861,21785,20264.0,38.41,56.04


In [6]:
df_data.columns

Index(['timestamp', 'generation biomass',
       'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual'],
      dtype='object')

In [7]:
list_columns = df_data.columns.str.replace(' ', '_').str.replace('-', '_').str.replace('/', '_')

In [8]:
list_columns

Index(['timestamp', 'generation_biomass',
       'generation_fossil_brown_coal_lignite',
       'generation_fossil_coal_derived_gas', 'generation_fossil_gas',
       'generation_fossil_hard_coal', 'generation_fossil_oil',
       'generation_fossil_oil_shale', 'generation_fossil_peat',
       'generation_geothermal', 'generation_hydro_pumped_storage_aggregated',
       'generation_hydro_pumped_storage_consumption',
       'generation_hydro_run_of_river_and_poundage',
       'generation_hydro_water_reservoir', 'generation_marine',
       'generation_nuclear', 'generation_other', 'generation_other_renewable',
       'generation_solar', 'generation_waste', 'generation_wind_offshore',
       'generation_wind_onshore', 'forecast_solar_day_ahead',
       'forecast_wind_offshore_eday_ahead', 'forecast_wind_onshore_day_ahead',
       'total_load_forecast', 'total_load_actual', 'price_day_ahead',
       'price_actual'],
      dtype='object')

In [9]:
df_data = df_data.rename(columns=dict(zip(df_data.columns, list_columns)))

In [10]:
list_generation_source = [
    'generation_biomass',
    'generation_fossil_brown_coal_lignite',
    'generation_fossil_coal_derived_gas', 'generation_fossil_gas',
    'generation_fossil_hard_coal', 'generation_fossil_oil',
    'generation_fossil_oil_shale', 'generation_fossil_peat',
    'generation_geothermal', 
    'generation_hydro_pumped_storage_aggregated',
    'generation_hydro_pumped_storage_consumption',
    'generation_hydro_run_of_river_and_poundage',
    'generation_hydro_water_reservoir', 
    'generation_marine',
    'generation_nuclear', 
    'generation_other', 
    'generation_other_renewable',
    'generation_solar', 
    'generation_waste', 
    'generation_wind_offshore',
    'generation_wind_onshore'
    ]

In [11]:
# df_data['timestamp'] = pd.to_datetime(df_data['timestamp'], utc=False)

ts = pd.to_datetime(df_data['timestamp'])

/tmp/ipykernel_529066/2576361507.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  ts = pd.to_datetime(df_data['timestamp'])


In [12]:
df_data.describe()

,generation_biomass,generation_fossil_brown_coal_lignite,generation_fossil_coal_derived_gas,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_fossil_oil_shale,generation_fossil_peat,generation_geothermal,generation_hydro_pumped_storage_aggregated,...,generation_waste,generation_wind_offshore,generation_wind_onshore,forecast_solar_day_ahead,forecast_wind_offshore_eday_ahead,forecast_wind_onshore_day_ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
count,35045.000000,35046.000000,35046.0,35046.000000,35046.000000,35045.000000,35046.0,35046.0,35046.0,0.0,...,35045.000000,35046.0,35046.000000,35064.000000,0.0,35064.000000,35064.000000,35028.000000,35064.000000,35064.000000
mean,383.513540,448.059208,0.0,5622.737488,4256.065742,298.319789,0.0,0.0,0.0,NaN,...,269.452133,0.0,5464.479769,1439.066735,NaN,5471.216689,28712.129962,28696.939905,49.874341,57.884023
std,85.353943,354.568590,0.0,2201.830478,1961.601013,52.520673,0.0,0.0,0.0,NaN,...,50.195536,0.0,3213.691587,1677.703355,NaN,3176.312853,4594.100854,4574.987950,14.618900,14.204083
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,...,0.000000,0.0,0.000000,0.000000,NaN,237.000000,18105.000000,18041.000000,2.060000,9.330000
25%,333.000000,0.000000,0.0,4126.000000,2527.000000,263.000000,0.0,0.0,0.0,NaN,...,240.000000,0.0,2933.000000,69.000000,NaN,2979.000000,24793.750000,24807.750000,41.490000,49.347500
50%,367.000000,509.000000,0.0,4969.000000,4474.000000,300.000000,0.0,0.0,0.0,NaN,...,279.000000,0.0,4849.000000,576.000000,NaN,4855.000000,28906.000000,28901.000000,50.520000,58.020000
75%,433.000000,757.000000,0.0,6429.000000,5838.750000,330.000000,0.0,0.0,0.0,NaN,...,310.000000,0.0,7398.000000,2636.000000,NaN,7353.000000,32263.250000,32192.000000,60.530000,68.010000
max,592.000000,999.000000,0.0,20034.000000,8359.000000,449.000000,0.0,0.0,0.0,NaN,...,357.000000,0.0,17436.000000,5836.000000,NaN,17430.000000,41390.000000,41015.000000,101.990000,116.800000


In [13]:
df_data.isna().sum()

timestamp                                          0
generation_biomass                                19
generation_fossil_brown_coal_lignite              18
generation_fossil_coal_derived_gas                18
generation_fossil_gas                             18
generation_fossil_hard_coal                       18
generation_fossil_oil                             19
generation_fossil_oil_shale                       18
generation_fossil_peat                            18
generation_geothermal                             18
generation_hydro_pumped_storage_aggregated     35064
generation_hydro_pumped_storage_consumption       19
generation_hydro_run_of_river_and_poundage        19
generation_hydro_water_reservoir                  18
generation_marine                                 19
generation_nuclear                                17
generation_other                                  18
generation_other_renewable                        18
generation_solar                              

In [14]:
df_data[list_generation_source] = df_data[list_generation_source].fillna(0)

In [15]:
df_data.isna().sum()

timestamp                                          0
generation_biomass                                 0
generation_fossil_brown_coal_lignite               0
generation_fossil_coal_derived_gas                 0
generation_fossil_gas                              0
generation_fossil_hard_coal                        0
generation_fossil_oil                              0
generation_fossil_oil_shale                        0
generation_fossil_peat                             0
generation_geothermal                              0
generation_hydro_pumped_storage_aggregated         0
generation_hydro_pumped_storage_consumption        0
generation_hydro_run_of_river_and_poundage         0
generation_hydro_water_reservoir                   0
generation_marine                                  0
generation_nuclear                                 0
generation_other                                   0
generation_other_renewable                         0
generation_solar                              

### Define KPI

In [16]:

df_formula = pd.DataFrame({'KPI':[], 'FORMULA':[]})
df_formula = df_formula.set_index('KPI')

print(df_formula)

Empty DataFrame
Columns: [FORMULA]
Index: []


In [17]:
test_str = "safediv(generation_fossil_brown_coal_lignite, generation_fossil_coal-derived_gas, generation_fossil_gas,\
    'generation_fossil_hard_coal', 'generation_fossil_oil','generation_fossil_oil_shale', 'generation_fossil_peat'"

In [18]:
print(test_str)

safediv(generation_fossil_brown_coal_lignite, generation_fossil_coal-derived_gas, generation_fossil_gas,    'generation_fossil_hard_coal', 'generation_fossil_oil','generation_fossil_oil_shale', 'generation_fossil_peat'


In [19]:
new_str = "'generation_biomass', 'generation_fossil_brown_coal_lignite', 'generation_fossil_coal-derived_gas', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_fossil_oil_shale', 'generation_fossil_peat', 'generation_geothermal',  'generation_hydro_pumped_storage_aggregated', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run-of-river_and_poundage',     'generation_hydro_water_reservoir',  'generation_marine', 'generation_nuclear',      'generation_other',  'generation_other_renewable',     'generation_solar',  'generation_waste',  'generation_wind_offshore', 'generation_wind_onshore'".replace("'","").replace(",", " +")

In [20]:
new_str

'generation_biomass + generation_fossil_brown_coal_lignite + generation_fossil_coal-derived_gas + generation_fossil_gas + generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat + generation_geothermal +  generation_hydro_pumped_storage_aggregated + generation_hydro_pumped_storage_consumption + generation_hydro_run-of-river_and_poundage +     generation_hydro_water_reservoir +  generation_marine + generation_nuclear +      generation_other +  generation_other_renewable +     generation_solar +  generation_waste +  generation_wind_offshore + generation_wind_onshore'

In [21]:
dict_num_den_kpi_formula = {
    "fossil_energy_ratio_num" : 
        "generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas +\
        generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat",
    "fossil_energy_ratio_den" : 
        "generation_biomass + generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas + generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat + generation_geothermal +  generation_hydro_pumped_storage_aggregated + generation_hydro_pumped_storage_consumption + generation_hydro_run_of_river_and_poundage +     generation_hydro_water_reservoir +  generation_marine + generation_nuclear +      generation_other +  generation_other_renewable +     generation_solar +  generation_waste +  generation_wind_offshore + generation_wind_onshore",
}

In [22]:
dict_final_kpi_formula = {
    "energy_total" : 
        "generation_biomass + generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas + generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat + generation_geothermal +  generation_hydro_pumped_storage_aggregated + generation_hydro_pumped_storage_consumption + generation_hydro_run_of_river_and_poundage +     generation_hydro_water_reservoir +  generation_marine + generation_nuclear +      generation_other +  generation_other_renewable +     generation_solar +  generation_waste +  generation_wind_offshore + generation_wind_onshore",
    "fossil_energy_ratio": 
        "safediv(fossil_energy_ratio_num, fossil_energy_ratio_den)"
}

In [23]:
dict_kpi_formula = {
    "energy_total" : 
        {
            "type": "normal",
            "formula": "generation_biomass + generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas + generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat + generation_geothermal +  generation_hydro_pumped_storage_aggregated + generation_hydro_pumped_storage_consumption + generation_hydro_run_of_river_and_poundage +     generation_hydro_water_reservoir +  generation_marine + generation_nuclear +      generation_other +  generation_other_renewable +     generation_solar +  generation_waste +  generation_wind_offshore + generation_wind_onshore",
        },
    "fossil_energy_ratio": 
        {
            "type": "ratio",
            "formula": "safediv(generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas +\
                        generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat,\
                        generation_biomass + generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas +\
                        generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat + generation_geothermal +\
                        generation_hydro_pumped_storage_aggregated + generation_hydro_pumped_storage_consumption + generation_hydro_run_of_river_and_poundage +\
                        generation_hydro_water_reservoir +  generation_marine + generation_nuclear + generation_other +  generation_other_renewable +\
                        generation_solar +  generation_waste +  generation_wind_offshore + generation_wind_onshore)"
        }        

}

In [24]:
extract_unique_metric_terms_from_formula(dict_kpi_formula["fossil_energy_ratio"]["formula"])

['generation_biomass',
 'generation_fossil_brown_coal_lignite',
 'generation_fossil_coal_derived_gas',
 'generation_fossil_gas',
 'generation_fossil_hard_coal',
 'generation_fossil_oil',
 'generation_fossil_oil_shale',
 'generation_fossil_peat',
 'generation_geothermal',
 'generation_hydro_pumped_storage_aggregated',
 'generation_hydro_pumped_storage_consumption',
 'generation_hydro_run_of_river_and_poundage',
 'generation_hydro_water_reservoir',
 'generation_marine',
 'generation_nuclear',
 'generation_other',
 'generation_other_renewable',
 'generation_solar',
 'generation_waste',
 'generation_wind_offshore',
 'generation_wind_onshore']

### Generating Base KPI

In [25]:
df_num_den_kpi = pd.DataFrame()

for key, item in dict_num_den_kpi_formula.items():

    df_num_den_kpi[key] = generate_kpi_series_multiprocess(item, df_data)
    


/home/blocktist/Projects/analytics/env_analytics/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/blocktist/Projects/analytics/env_analytics/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [26]:
df_num_den_kpi

,fossil_energy_ratio_num,fossil_energy_ratio_den
0,10156.0,28251.0
1,10437.0,27818.0
2,9918.0,26796.0
3,8859.0,25223.0
4,8313.0,24620.0
...,...,...
35059,10440.0,26415.0
35060,9981.0,25213.0
35061,9615.0,23987.0
35062,9018.0,22748.0


In [27]:
dict_final_kpi_formula

{'energy_total': 'generation_biomass + generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas + generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat + generation_geothermal +  generation_hydro_pumped_storage_aggregated + generation_hydro_pumped_storage_consumption + generation_hydro_run_of_river_and_poundage +     generation_hydro_water_reservoir +  generation_marine + generation_nuclear +      generation_other +  generation_other_renewable +     generation_solar +  generation_waste +  generation_wind_offshore + generation_wind_onshore',
 'fossil_energy_ratio': 'safediv(fossil_energy_ratio_num, fossil_energy_ratio_den)'}

In [28]:
dict_kpi_formula

{'energy_total': {'type': 'normal',
  'formula': 'generation_biomass + generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas + generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat + generation_geothermal +  generation_hydro_pumped_storage_aggregated + generation_hydro_pumped_storage_consumption + generation_hydro_run_of_river_and_poundage +     generation_hydro_water_reservoir +  generation_marine + generation_nuclear +      generation_other +  generation_other_renewable +     generation_solar +  generation_waste +  generation_wind_offshore + generation_wind_onshore'},
 'fossil_energy_ratio': {'type': 'ratio',
  'formula': 'safediv(generation_fossil_brown_coal_lignite + generation_fossil_coal_derived_gas + generation_fossil_gas +                        generation_fossil_hard_coal + generation_fossil_oil + generation_fossil_oil_shale + generation_fossil_peat,                        generati

In [29]:
df_data

,timestamp,generation_biomass,generation_fossil_brown_coal_lignite,generation_fossil_coal_derived_gas,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_fossil_oil_shale,generation_fossil_peat,generation_geothermal,...,generation_waste,generation_wind_offshore,generation_wind_onshore,forecast_solar_day_ahead,forecast_wind_offshore_eday_ahead,forecast_wind_onshore_day_ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17,NaN,6436,26118,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16,NaN,5856,24934,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8,NaN,5454,23515,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2,NaN,5151,22642,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9,NaN,4861,21785,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,277.0,0.0,3113.0,96,NaN,3253,30619,30653.0,68.85,77.02
35060,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,280.0,0.0,3288.0,51,NaN,3353,29932,29735.0,68.40,76.16
35061,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,286.0,0.0,3503.0,36,NaN,3404,27903,28071.0,66.88,74.30
35062,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,287.0,0.0,3586.0,29,NaN,3273,25450,25801.0,63.93,69.89


In [30]:
df_kpi = pd.DataFrame()

for key, item in dict_kpi_formula.items():
    # formula = item
    kpi_type = item['type']
    str_formula = item['formula']
    # print('str_formula:', str_formula)

    if kpi_type == 'normal':
        df_kpi[key] = generate_kpi_series_multiprocess(str_formula, df_data)    

    elif kpi_type == 'ratio':
        # continue
        # df_kpi[key] = generate_kpi_series_multiprocess(str_formula, df_num_den_kpi)
        df_kpi[key] = generate_kpi_series_multiprocess(str_formula, df_data)
        

/home/blocktist/Projects/analytics/env_analytics/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/blocktist/Projects/analytics/env_analytics/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [31]:
df_kpi

,energy_total,fossil_energy_ratio
0,28251.0,0.359492
1,27818.0,0.375189
2,26796.0,0.370130
3,25223.0,0.351227
4,24620.0,0.337652
...,...,...
35059,26415.0,0.395230
35060,25213.0,0.395867
35061,23987.0,0.400842
35062,22748.0,0.396430


### Generating Final KPI

In [36]:
df_kpi

,energy_total,fossil_energy_ratio
0,28251.0,0.359492
1,27818.0,0.375189
2,26796.0,0.370130
3,25223.0,0.351227
4,24620.0,0.337652
...,...,...
35059,26415.0,0.395230
35060,25213.0,0.395867
35061,23987.0,0.400842
35062,22748.0,0.396430


In [33]:
df_data_and_kpi = pd.concat([df_data, df_kpi], axis=1)

In [34]:
df_data_and_kpi

,timestamp,generation_biomass,generation_fossil_brown_coal_lignite,generation_fossil_coal_derived_gas,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_fossil_oil_shale,generation_fossil_peat,generation_geothermal,...,generation_wind_onshore,forecast_solar_day_ahead,forecast_wind_offshore_eday_ahead,forecast_wind_onshore_day_ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual,energy_total,fossil_energy_ratio
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,6378.0,17,NaN,6436,26118,25385.0,50.10,65.41,28251.0,0.359492
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,5890.0,16,NaN,5856,24934,24382.0,48.10,64.92,27818.0,0.375189
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,5461.0,8,NaN,5454,23515,22734.0,47.33,64.48,26796.0,0.370130
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,5238.0,2,NaN,5151,22642,21286.0,42.27,59.32,25223.0,0.351227
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,4935.0,9,NaN,4861,21785,20264.0,38.41,56.04,24620.0,0.337652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,3113.0,96,NaN,3253,30619,30653.0,68.85,77.02,26415.0,0.395230
35060,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,3288.0,51,NaN,3353,29932,29735.0,68.40,76.16,25213.0,0.395867
35061,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,3503.0,36,NaN,3404,27903,28071.0,66.88,74.30,23987.0,0.400842
35062,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,3586.0,29,NaN,3273,25450,25801.0,63.93,69.89,22748.0,0.396430


In [35]:
target_kpi = 'fossil_energy_ratio'
list_terms = extract_terms(dict_final_kpi_formula[target_kpi])
if 'safediv' in list_terms:
    list_terms.remove('safediv')


df_contribution = generate_contribution_series_multiprocess(target_kpi, list_terms, df_data_and_kpi)
df_contribution.columns = [target_kpi + '::' + term for term in list_terms]
df_contribution = pd.concat([df_data_and_kpi['timestamp'], df_contribution], axis=1)

/home/blocktist/Projects/analytics/env_analytics/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


KeyError: 'fossil_energy_ratio_num'

In [ ]:
df_contribution

In [ ]:


df_corr_input = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1).drop('timestamp', axis=1)

In [ ]:
df_corr_input.corr()

In [ ]:
df_corr = df_data_and_kpi.drop(['timestamp'] + list_generation_source, axis=1).corr()

In [ ]:
df_corr

In [ ]:
df_data_and_kpi['timestamp'].iloc[0]

In [ ]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# def plotly_dual_axis(df,  x="timestamp", y1="", y2="", title="", x_title="Time", y1_title="", y2_title="", different_y_axes_scale=True):

#     if y1_title == "":
#         y1_title = y1
#     if y2_title == "":
#         y2_title = y2

#     y1_label=y1
#     y2_label=y2

#     # Create figure with secondary y-axis
#     fig = make_subplots(specs=[[{"secondary_y": True}]])

#     # Add traces
#     fig.add_trace(
#         go.Scatter(
#             x=df[x], 
#             y=df[y1], 
#             hovertemplate = '[label]<br>Timestamp: %{x|%Y-%m-%d %H:%M:%S}<br>Value: %{y}<extra></extra>'.replace('[label]', y1_label),
#             name=y1_label),
#         secondary_y=False,
#     )

#     fig.add_trace(
#         go.Scatter(
#             x=df[x],
#             y=df[y2],
#             hovertemplate = '[label]<br>Timestamp: %{x|%Y-%m-%d %H:%M:%S}<br>Value: %{y}<extra></extra>'.replace('[label]', y2_label),
#             name=y2_label),
#         secondary_y=different_y_axes_scale,
#     )

#     # Add figure title
#     fig.update_layout(
#         title_text=title
#     )

#     # Set x-axis title
#     fig.update_xaxes(title_text=x_title)

#     # Set y-axes titles
#     fig.update_yaxes(title_text=y1_title, secondary_y=False)
#     if different_y_axes_scale==True:
#         fig.update_yaxes(title_text=y2_title, secondary_y=True)

#     # fig.show()



#     return fig


# def plotly_dual_axis_multi_y2(df,  x="timestamp", y1="", list_y2=[], title="", x_title="Time", y1_title="", y2_title="", different_y_axes_scale=True, scaler=None):

#     if y1_title == "":
#         y1_title = y1


#     # Create figure with secondary y-axis
#     fig = make_subplots(specs=[[{"secondary_y": True}]])

#     # Add traces
    
#     y1_label=y1
#     # fig.add_trace(
#     #     go.Scatter(mode='lines+markers', x=df[x], y=df[y1], name=y1_label),
#     #     secondary_y=False,
#     # )
#     fig.add_trace(
#         go.Scatter(
#             mode='lines+markers',
#             x=df[x].values, 
#             y=df[y1].values, 
#             hovertemplate = "[label]<br>Timestamp: %{x|%Y-%m-%d %H:%M:%S}<br>Value: %{y}<extra></extra>".replace('[label]', y1_label),
#             name=y1_label),
#         secondary_y=False,
#     )

#     for y2 in list_y2:
#         y2_label=y2
        
#         if scaler == None:
#             y_series = df[y2].values
#         else:
#             if scaler == 'minmax':
#                 y_series = MinMaxScaler().fit_transform(pd.DataFrame(df[y2])).reshape(-1, )
#             elif scaler == 'standard':
#                 y_series = StandardScaler().fit_transform(pd.DataFrame(df[y2])).reshape(-1, )
#             else:
#                 y_series = df[y2].values
#                 print('#### Wrong normalization setting!')

#         fig.add_trace(
#             go.Scatter(
#                 mode='lines',
#                 x=df[x], 
#                 y=y_series, 
#                 hovertemplate = "[label]<br>Timestamp: %{x|%Y-%m-%d %H:%M:%S}<br>Value: %{y}<extra></extra>".replace('[label]', y2_label),
#                 name=y2_label),
#             secondary_y=different_y_axes_scale,
#         )


#     # Add figure title
#     fig.update_layout(
#         title_text=title
#     )

#     # Set x-axis title
#     fig.update_xaxes(title_text=x_title)

#     # Set y-axes titles
    
#     fig.update_yaxes(title_text=y1_title, secondary_y=False)
#     if different_y_axes_scale == True:
#         fig.update_yaxes(title_text=y2_title, secondary_y=True)

#     # fig.show()

#     return fig




# def plotly_multi_timeseries(df,  x="timestamp", list_y=[], title="", x_title="Time", y_title="", scaler=None):


#     # Create figure with secondary y-axis
#     fig = make_subplots(specs=[[{"secondary_y": False}]])

#     # Add traces

#     # fig.add_trace(
#     #     go.Scatter(mode='lines+markers', x=df[x], y=df[y1], name=y1_label),
#     #     secondary_y=False,
#     # )
#     # fig.add_trace(
#     #     go.Scatter(
#     #         mode='lines+markers',
#     #         x=df[x], 
#     #         y=df[y1], 
#     #         hovertemplate = "[label]<br>Timestamp: %{x|%Y-%m-%d %H:%M:%S}<br>Value: %{y}<extra></extra>".replace('[label]', y1_label),
#     #         name=y1_label),
#     #     secondary_y=False,
#     # )



#     for y in list_y:
#         y_label=y
#         # fig.add_trace(
#         #     go.Scatter(mode='lines', x=df[x], y=df[y2], name=y2_label),
#         #     secondary_y=True,
#         # )
#         if scaler == None:
#             y_series = df[y].values
#         else:
#             if scaler == 'minmax':
#                 y_series = MinMaxScaler().fit_transform(pd.DataFrame(df[y])).reshape(-1, )
#             elif scaler == 'standard':
#                 y_series = StandardScaler().fit_transform(pd.DataFrame(df[y])).reshape(-1, )
#             else:
#                 y_series = df[y].values
#                 print('#### Wrong normalization setting!')

#         fig.add_trace(
#             go.Scatter(
#                 mode='lines',
#                 x=df[x].values, 
#                 y=y_series, 
#                 hovertemplate = "[label]<br>Timestamp: %{x|%Y-%m-%d %H:%M:%S}<br>Value: %{y}<extra></extra>".replace('[label]', y_label),
#                 name=y_label)
#         )


#     # Add figure title
#     fig.update_layout(
#         title_text=title
#     )

#     # Set x-axis title
#     fig.update_xaxes(title_text=x_title)

#     # Set y-axes titles
    
#     fig.update_yaxes(title_text=y_title, secondary_y=False)

#     # fig.show()


#     return fig

In [ ]:
start_time = pd.to_datetime('2018-01-01').tz_localize('UTC')
end_time = pd.to_datetime('2018-03-31').tz_localize('UTC')

df_plot = df_data_and_kpi[
    (df_data_and_kpi['timestamp'] >= start_time) & (df_data_and_kpi['timestamp'] <= end_time)
]

In [ ]:
df_plot.columns

In [ ]:
y_series = MinMaxScaler().fit_transform(pd.DataFrame(df_plot['generation_biomass'])).reshape(-1, )

In [ ]:
y_series

In [ ]:
plotly_dual_axis_multi_y2(df_plot,  x="timestamp", list_y=['fossil_energy_ratio_num', 'generation_fossil_brown_coal_lignite',
       'generation_fossil_coal_derived_gas', 'generation_fossil_gas',
       'generation_fossil_hard_coal', 'generation_fossil_oil'], title="", x_title="Time", y_title="", scaler=None)

In [ ]:
plotly_multi_timeseries(df_plot,  x="timestamp", list_y=['fossil_energy_ratio_num', 'generation_fossil_brown_coal_lignite',
       'generation_fossil_coal_derived_gas', 'generation_fossil_gas',
       'generation_fossil_hard_coal', 'generation_fossil_oil'], title="", x_title="Time", y_title="", scaler=None)

In [ ]:
plotly_multi_timeseries(df_plot,  x="timestamp", list_y=['generation_fossil_brown_coal_lignite',
       'generation_fossil_coal_derived_gas', 'generation_fossil_gas',
       'generation_fossil_hard_coal', 'generation_fossil_oil'], title="", x_title="Time", y_title="", scaler='minmax')

In [ ]:
plotly_multi_timeseries(df_plot,  x="timestamp", list_y=['generation_fossil_brown_coal_lignite',
       'generation_fossil_coal_derived_gas', 'generation_fossil_gas',
       'generation_fossil_hard_coal', 'generation_fossil_oil'], title="", x_title="Time", y_title="", scaler='standard')

In [ ]:

start_time = pd.to_datetime('2018-01-01').tz_localize('UTC')
end_time = pd.to_datetime('2018-03-31').tz_localize('UTC')

df_plot = df_data_and_kpi[
    (df_data_and_kpi['timestamp'] >= start_time) & (df_data_and_kpi['timestamp'] <= end_time)
]

In [ ]:
plotly_dual_axis(df_plot, x='timestamp', y1='energy_total', y2='generation_fossil_brown_coal_lignite', different_y_axes_scale=False)

### Final KPI

In [ ]:
target_kpi = 'fossil_energy_ratio'


--> No usable of contribution for final KPI

In [ ]:
dict_final_kpi_formula

In [ ]:
target_kpi = 'fossil_energy_ratio'
list_terms = extract_terms(dict_final_kpi_formula[target_kpi])
if 'safediv' in list_terms:
    list_terms.remove('safediv')



In [ ]:
# df_plot = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1)

df_plot = df_data_and_kpi.copy()

# start_time = pd.to_datetime('2018-01-25').tz_localize('UTC')
# end_time = pd.to_datetime('2018-01-31').tz_localize('UTC')

start_time = pd.to_datetime('2018-01-31').tz_localize('UTC')
end_time = pd.to_datetime('2018-02-10').tz_localize('UTC')

df_plot = df_plot[
    (df_plot['timestamp'] >= start_time) & (df_plot['timestamp'] <= end_time)
]

In [ ]:
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_terms, y2_title='energy', different_y_axes_scale=True)

--> fossil_energy_den  need to be analyzed

### Num/Den Analysis

In [ ]:
target_kpi = 'fossil_energy_ratio_num'
list_terms = extract_terms(dict_num_den_kpi_formula[target_kpi])


df_contribution = generate_contribution_series_multiprocess(target_kpi, list_terms, df_data_and_kpi)
list_new_terms = [target_kpi + '::' + term for term in list_terms]
df_contribution.columns = list_new_terms
df_contribution = pd.concat([df_data_and_kpi['timestamp'], df_contribution], axis=1)

df_plot = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1)

start_time = pd.to_datetime('2018-01-31').tz_localize('UTC')
end_time = pd.to_datetime('2018-02-10').tz_localize('UTC')

df_plot = df_plot[
    (df_plot['timestamp'] >= start_time) & (df_plot['timestamp'] <= end_time)
]
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_new_terms, y2_title='contribution', different_y_axes_scale=True)

In [ ]:
target_kpi = 'fossil_energy_ratio_num'
list_terms = extract_terms(dict_num_den_kpi_formula[target_kpi])

df_plot = df_data_and_kpi[['timestamp', target_kpi] + list_terms]

start_time = pd.to_datetime('2018-01-31').tz_localize('UTC')
end_time = pd.to_datetime('2018-02-10').tz_localize('UTC')

df_plot = df_plot[
    (df_plot['timestamp'] >= start_time) & (df_plot['timestamp'] <= end_time)
]
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_terms, y2_title='energy', different_y_axes_scale=False)

#### Correlation check

In [ ]:
target_kpi = 'fossil_energy_ratio'

start_time = pd.to_datetime('2018-01-31').tz_localize('UTC')
end_time = pd.to_datetime('2018-02-10').tz_localize('UTC')

df_analysis = df_data_and_kpi[(df_data_and_kpi['timestamp'] >= start_time) & (df_data_and_kpi['timestamp'] <= end_time)]

mertic_name, corr_coeff = indentify_correlated_metric_term_with_kpi(df_analysis, target_kpi, dict_kpi_formula)

In [ ]:
mertic_name, corr_coeff

In [ ]:
plotly_dual_axis_multi_y2(df_analysis, x='timestamp', y1=target_kpi, list_y2=list_terms, y2_title='energy', different_y_axes_scale=True)

In [ ]:
df_analysis[[target_kpi] + list_terms].corr()

In [ ]:
mertic_name, corr_coeff 

In [ ]:

df_plot = pd.concat([df_data_and_kpi[['timestamp', target_kpi] + list_terms], df_contribution], axis=1)  


In [ ]:
df_plot

In [ ]:
get_highest_corr_ts(df_plot, target_kpi, list_terms + list_new_terms, absolute=False)

In [ ]:
get_highest_corr_ts(df_plot, target_kpi, list_terms, absolute=False)

In [ ]:
get_highest_corr_ts(df_plot, target_kpi, list_terms, absolute=False, corr_method='spearman')

In [ ]:
get_highest_corr_ts(df_plot, target_kpi, list_terms, absolute=False, corr_method='kendall')

In [ ]:
get_highest_corr_ts(df_plot, target_kpi, list_terms, absolute=False)

In [ ]:
target_kpi = 'fossil_energy_ratio_den'
list_terms = extract_terms(dict_num_den_kpi_formula[target_kpi])


df_contribution = generate_contribution_series_multiprocess(target_kpi, list_terms, df_data_and_kpi)
list_new_terms = [target_kpi + '::' + term for term in list_terms]
df_contribution.columns = list_new_terms
df_contribution = pd.concat([df_data_and_kpi['timestamp'], df_contribution], axis=1)
df_plot = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1)

start_time = pd.to_datetime('2018-01-31').tz_localize('UTC')
end_time = pd.to_datetime('2018-02-10').tz_localize('UTC')

df_plot = df_plot[
    (df_plot['timestamp'] >= start_time) & (df_plot['timestamp'] <= end_time)
]
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_new_terms, y2_title='contribution', different_y_axes_scale=True)

In [ ]:
target_kpi = 'fossil_energy_ratio_den'
list_terms = extract_terms(dict_num_den_kpi_formula[target_kpi])


df_contribution = generate_contribution_series_multiprocess(target_kpi, list_terms, df_data_and_kpi)
list_new_terms = [target_kpi + '::' + term for term in list_terms]
df_contribution.columns = list_new_terms
df_contribution = pd.concat([df_data_and_kpi['timestamp'], df_contribution], axis=1)

In [ ]:
df_plot = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1)

start_time = pd.to_datetime('2018-01-31').tz_localize('UTC')
end_time = pd.to_datetime('2018-02-10').tz_localize('UTC')

df_plot = df_plot[
    (df_contribution['timestamp'] >= start_time) & (df_contribution['timestamp'] <= end_time)
]

In [ ]:
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_new_terms, y2_title='contribution', different_y_axes_scale=True)

In [ ]:
df_plot

In [ ]:
get_highest_corr_ts(df_plot, target_kpi, list_new_terms, absolute=False)

In [ ]:
df_plot = df_data_and_kpi.copy()

start_time = pd.to_datetime('2018-01-31').tz_localize('UTC')
end_time = pd.to_datetime('2018-02-10').tz_localize('UTC')

df_plot = df_plot[
    (df_contribution['timestamp'] >= start_time) & (df_contribution['timestamp'] <= end_time)
]

In [ ]:
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_terms, y2_title='energy')

In [ ]:
target_kpi = 'fossil_energy_ratio_num'
list_terms = extract_terms(dict_num_den_kpi_formula[target_kpi])

df_contribution = generate_contribution_series_multiprocess(target_kpi, list_terms, df_data_and_kpi)
list_new_terms = [target_kpi + '::' + term for term in list_terms]
df_contribution.columns = list_new_terms
df_contribution = pd.concat([df_data_and_kpi['timestamp'], df_contribution], axis=1)

In [ ]:
df_plot = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1)

start_time = pd.to_datetime('2018-01-01').tz_localize('UTC')
end_time = pd.to_datetime('2018-03-31').tz_localize('UTC')

df_plot = df_plot[
    (df_contribution['timestamp'] >= start_time) & (df_contribution['timestamp'] <= end_time)
]

In [ ]:
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_new_terms, y2_title='contribution')

In [ ]:
target_kpi = 'fossil_energy_ratio_num'
list_terms = extract_terms(dict_num_den_kpi_formula[target_kpi])

df_contribution = generate_contribution_series_multiprocess(target_kpi, list_terms, df_data_and_kpi)
list_new_terms = [target_kpi + '::' + term for term in list_terms]
df_contribution.columns = list_new_terms
df_contribution = pd.concat([df_data_and_kpi['timestamp'], df_contribution], axis=1)

In [ ]:
df_plot = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1)

start_time = pd.to_datetime('2018-01-01').tz_localize('UTC')
end_time = pd.to_datetime('2018-03-31').tz_localize('UTC')

df_plot = df_plot[
    (df_contribution['timestamp'] >= start_time) & (df_contribution['timestamp'] <= end_time)
]

In [ ]:
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_new_terms, y2_title='contribution')

In [ ]:
# df_plot = pd.concat([df_data_and_kpi[target_kpi], df_contribution], axis=1)

df_plot = df_data_and_kpi.copy()

# start_time = pd.to_datetime('2018-01-25').tz_localize('UTC')
# end_time = pd.to_datetime('2018-01-31').tz_localize('UTC')

df_plot = df_plot[
    (df_plot['timestamp'] >= start_time) & (df_plot['timestamp'] <= end_time)
]

In [ ]:
df_plot[['timestamp', target_kpi] + list_terms]

In [ ]:
plotly_dual_axis_multi_y2(df_plot, x='timestamp', y1=target_kpi, list_y2=list_terms, y2_title='energy')

In [ ]:
plot_col = 'energy_total'
px.line(df_plot, x='timestamp',  y=plot_col)

In [ ]:

plot_col = 'total_load_forecast'
px.line(df_plot, x='timestamp',  y=plot_col)

In [ ]:

plot_col = 'total_load_actual'
px.line(df_plot, x='timestamp',  y=plot_col)

In [ ]:
plot_col = 'price_actual'
px.line(df_plot, x='timestamp',  y=plot_col)

In [ ]:
df_data_and_kpi.head()